In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import matplotlib 
from sklearn.model_selection import cross_val_score

In [5]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import ShuffleSplit
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor
from sklearn.metrics import mean_squared_error

In [7]:
df1=pd.read_csv('bangalore_house_data_cleaned.csv')
df1.head()

,area_type,location,size,total_sqft,bath,balcony,price,no_of_bedrooms,price_per_sqft
0,Super built-up Area,1st Block Jayanagar,4 BHK,2850.0,4.0,1.0,428.0,4,15017.543860
1,Super built-up Area,1st Block Jayanagar,3 BHK,1630.0,3.0,2.0,194.0,3,11901.840491
2,Super built-up Area,1st Block Jayanagar,3 BHK,1875.0,2.0,3.0,235.0,3,12533.333333
3,Built-up Area,1st Block Jayanagar,3 BHK,1200.0,2.0,0.0,130.0,3,10833.333333
4,Super built-up Area,1st Block Jayanagar,2 BHK,1235.0,2.0,2.0,148.0,2,11983.805668


In [8]:
df2=df1.drop(['size','price_per_sqft'],axis=1)
df2.head()

,area_type,location,total_sqft,bath,balcony,price,no_of_bedrooms
0,Super built-up Area,1st Block Jayanagar,2850.0,4.0,1.0,428.0,4
1,Super built-up Area,1st Block Jayanagar,1630.0,3.0,2.0,194.0,3
2,Super built-up Area,1st Block Jayanagar,1875.0,2.0,3.0,235.0,3
3,Built-up Area,1st Block Jayanagar,1200.0,2.0,0.0,130.0,3
4,Super built-up Area,1st Block Jayanagar,1235.0,2.0,2.0,148.0,2


In [9]:
df2.corr()

,total_sqft,bath,balcony,price,no_of_bedrooms
total_sqft,1.000000,0.096212,0.020206,0.127317,0.092890
bath,0.096212,1.000000,0.237924,0.576978,0.855951
balcony,0.020206,0.237924,1.000000,0.164610,0.244115
price,0.127317,0.576978,0.164610,1.000000,0.526255
no_of_bedrooms,0.092890,0.855951,0.244115,0.526255,1.000000


In [10]:
df3=pd.get_dummies(df2)
df3

,total_sqft,bath,balcony,price,no_of_bedrooms,area_type_Built-up Area,area_type_Carpet Area,area_type_Plot Area,area_type_Super built-up Area,location_1st Block Jayanagar,...,location_Vishveshwarya Layout,location_Vishwapriya Layout,location_Vittasandra,location_Whitefield,location_Yelachenahalli,location_Yelahanka,location_Yelahanka New Town,location_Yelenahalli,location_Yeshwanthpur,location_other
0,2850.0,4.0,1.0,428.00,4,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1,1630.0,3.0,2.0,194.00,3,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
2,1875.0,2.0,3.0,235.00,3,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,1200.0,2.0,0.0,130.00,3,1,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,1235.0,2.0,2.0,148.00,2,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10306,1353.0,2.0,2.0,110.00,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
10307,812.0,1.0,0.0,26.00,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
10308,1440.0,2.0,2.0,63.93,3,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
10309,1075.0,2.0,2.0,48.00,2,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1


In [11]:
# get the values for axis ----- categories on x and price on y
x=df3.drop(['price'],axis=1)
y=df3.price

In [12]:
# Split data into train and test
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2,random_state=42)

In [13]:
# multiple linear regression
from sklearn import linear_model
from sklearn.metrics import mean_absolute_error
ml_regr = linear_model.LinearRegression()
ml_regr.fit(x_train, y_train)
print('mean error for gbr: ',np.mean(cross_val_score(ml_regr,x_train,y_train, scoring = 'neg_mean_absolute_error', cv= 3)))
ml_pred = ml_regr.predict(x_test)
print('MAE for ml >',mean_absolute_error(y_test,ml_pred))

mean error for gbr:  -30.597596733300616
MAE for ml > 28.76020656754376


In [14]:
from sklearn import ensemble
gbr_reg = ensemble.GradientBoostingRegressor(n_estimators = 400, max_depth = 5, min_samples_split = 2,learning_rate = 0.1, loss = 'ls')
gbr_reg.fit(x_train, y_train)
print('mean error for gbr: ',np.mean(cross_val_score(gbr_reg,x_train,y_train, scoring = 'neg_mean_absolute_error', cv= 3)))

mean error for gbr:  -17.974655718600395


In [15]:
GBR_pred = gbr_reg.predict(x_test)
print('MAE for GBR >',mean_absolute_error(y_test,GBR_pred))

MAE for GBR > 17.060877827189415


In [16]:
gbr = GradientBoostingRegressor(random_state=0)
param_grid = {
        'n_estimators': [500],
         'max_features':['auto', 'sqrt', 'log2'],
        'max_depth': [6,8,10],
        'learning_rate': [0.05,0.1,0.15] }
model_gbr = GridSearchCV(estimator=gbr, param_grid=param_grid, n_jobs=1, cv=3, scoring='neg_mean_absolute_error')
model_gbr.fit(x_train, y_train)
print('Gradient boosted tree regression...')
print('Best Params:')
print(model_gbr.best_params_)
print('Best CV Score:')
print(-model_gbr.best_score_)
print('mean error for : ',np.mean(cross_val_score(model_gbr.best_estimator_,x_train,y_train, scoring = 'neg_mean_absolute_error', cv= 3)))

Gradient boosted tree regression...
Best Params:
{'learning_rate': 0.1, 'max_depth': 10, 'max_features': 'sqrt', 'n_estimators': 500}
Best CV Score:
17.05724147936173
mean error for :  -17.05724147936173


In [17]:
GB_pred = model_gbr.best_estimator_.predict(x_test)
print('MAE for GB after tuning >',mean_absolute_error(y_test,GB_pred))

MAE for GB after tuning > 16.25662958520614


In [18]:
etr = ExtraTreesRegressor(n_jobs=1, random_state=0)
param_grid = {'n_estimators': [500], 'max_features':['auto','sqrt','log2']}
model_etr = GridSearchCV(estimator=etr, param_grid=param_grid, n_jobs=1, cv=3, scoring='neg_mean_absolute_error')
model_etr.fit(x_train, y_train)
print('Extra trees regression...')
print('Best Params:')
print(model_etr.best_params_)
print('Best CV Score:')
print(model_etr.best_score_)
print('mean error for rfr: ',np.mean(cross_val_score(model_etr.best_estimator_,x_train,y_train, scoring = 'neg_mean_absolute_error', cv= 3)))

Extra trees regression...
Best Params:
{'max_features': 'auto', 'n_estimators': 500}
Best CV Score:
-18.412820019118794
mean error for rfr:  -18.412820019118794


In [19]:
EXT_pred =model_etr.best_estimator_.predict(x_test)
print('MAE for EXT >',mean_absolute_error(y_test,EXT_pred))

MAE for EXT > 18.063123521329533


In [20]:
def predict_price1(area_type,location,sqft,bath,bhk,balcony):    
    loc_index = np.where(x.columns==location)[0][0]
    area_index= np.where(x.columns==area_type)[0][0]

    X = np.zeros(len(x.columns))
    X[0] = sqft
    X[1] = bath
    X[3] = bhk
    X[2]= balcony
    if loc_index >= 0:
        X[loc_index] = 1
    if area_index>=0:
        X[area_index] = 1
   

    return model_gbr.best_estimator_.predict([X])[0]

In [21]:
# area_type, location,sqft, bathroom, bedroom, balcony
predict_price1('area_type_Super built-up  Area','location_1st Block Jayanagar',1875,2,3,3)

234.5319768235101

In [24]:
predict_price1('area_type_Super built-up  Area','location_1st Block Jayanagar',1235,2,2,2)

147.65325180101024

In [26]:
df2.head()

,area_type,location,total_sqft,bath,balcony,price,no_of_bedrooms
0,Super built-up Area,1st Block Jayanagar,2850.0,4.0,1.0,428.0,4
1,Super built-up Area,1st Block Jayanagar,1630.0,3.0,2.0,194.0,3
2,Super built-up Area,1st Block Jayanagar,1875.0,2.0,3.0,235.0,3
3,Built-up Area,1st Block Jayanagar,1200.0,2.0,0.0,130.0,3
4,Super built-up Area,1st Block Jayanagar,1235.0,2.0,2.0,148.0,2


In [27]:
df3.columns

Index(['total_sqft', 'bath', 'balcony', 'price', 'no_of_bedrooms',
       'area_type_Built-up  Area', 'area_type_Carpet  Area',
       'area_type_Plot  Area', 'area_type_Super built-up  Area',
       'location_1st Block Jayanagar',
       ...
       'location_Vishveshwarya Layout', 'location_Vishwapriya Layout',
       'location_Vittasandra', 'location_Whitefield',
       'location_Yelachenahalli', 'location_Yelahanka',
       'location_Yelahanka New Town', 'location_Yelenahalli',
       'location_Yeshwanthpur', 'location_other'],
      dtype='object', length=251)

In [28]:
import pickle
with open('banglore_home_prices_model.pickle','wb') as f:
    pickle.dump(model_gbr.best_estimator_,f)

In [29]:
import json
columns = {
    'data_columns' : [col.lower() for col in x.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))